In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import smtplib
import os
from dotenv import load_dotenv
from pathlib import Path 


In [ ]:
# function to send email of cheap flight
def send_email(price,url):
    
        # create message object 
        msg = MIMEMultipart()
        
        message = "Low price found at: $" + price + "\nLINK: " + url

        env_path = Path('.')/'.env'
        load_dotenv(dotenv_path = env_path)
        
        # setup the parameters from env var
        password = os.getenv('PASSWORDEMAIL')
        msg['From'] = os.getenv('FROM')
        msg['To'] = os.getenv('TO')
        msg['Subject'] = "Price Match Found for Flight!"
        
        # add message 
        msg.attach(MIMEText(message, 'plain'))
         
        #create server
        server = smtplib.SMTP('smtp.gmail.com: 587')
        
        server.starttls()
        
        server.login(msg['From'], password)
        
        # send the message to email
        server.sendmail(msg['From'], msg['To'], msg.as_string())
        
        server.quit()


In [ ]:
while True:
    # open up google 
    driver=webdriver.Chrome()

    # store information for flight
    origin = 'ORD'
    destination = 'BWI'
    departure_date = '2024-12-14'
    return_date = '2024-12-20'


    # get to the website
    url = f'https://www.kayak.com/flights/{origin}-{destination}/{departure_date}/{return_date}?sort=price_a&fs=stops=-2'
    driver.get(url)
    time.sleep(15)
    content=driver.page_source
    soup=BeautifulSoup(content)

    for div in soup.findAll('div',attrs={'class':'Hv20-value'}):
        # gets the best price 
        price=div.text[1:4]

        if int(price) < 375:
            send_email(price,url)

    driver.quit()
    # time.sleep(10800)